In [ ]:
import streamlit as st
import numpy as np
from tensorflow.keras.models import load_model
import librosa
import queue
import sounddevice as sd

# Load the pre-trained model
model = load_model('')

# Streamlit app
st.title('Real-time Sound Classification for Conservation')

# Create a queue to store incoming sound data
sound_queue = queue.Queue(maxsize=44100)  # Queue size for one second of audio at 44.1 kHz

# Function to process audio stream
def callback(indata, frames, time, status):
    if status:
        print(status, flush=True)
    sound_queue.put(indata.copy())

# Open a streaming audio input
with sd.InputStream(callback=callback):
    while True:
        # Read sound data from the queue
        sound_data = sound_queue.get()

        # Display the streaming audio
        st.audio(sound_data, format='audio/wav', start_time=0)

        # Process the sound data for classification
        sound_features = extract_features(sound_data)

        # Make a prediction
        prediction = model.predict_classes(np.expand_dims(sound_features, axis=0))[0]

        # Reverse the label encoding
        predicted_class = label_encoder.inverse_transform([prediction])[0]

        # Display the result
        st.write(f'Prediction: {predicted_class}')